In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://pokemondb.net"
POKEDEX_URL = BASE_URL + "/pokedex/national"

STATS = ["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]

# Mapping for Pokémon with special URL slugs
SPECIAL_NAMES = {
    "Nidoran♀": "nidoran-f",
    "Nidoran♂": "nidoran-m",
    "Farfetch'd": "farfetchd",
    "Mr. Mime": "mr-mime",
    "Mr. Rime": "mr-rime",
    "Type: Null": "type-null",
    "Tapu Koko": "tapu-koko",
    "Tapu Lele": "tapu-lele",
    "Tapu Bulu": "tapu-bulu",
    "Tapu Fini": "tapu-fini",
    "Flabébé": "flabebe",
}

def get_all_pokemon_names():
    response = requests.get(POKEDEX_URL)
    if response.status_code != 200:
        raise Exception("Failed to fetch Pokédex page")
    
    soup = BeautifulSoup(response.text, "html.parser")
    pokemon_links = soup.select("a.ent-name")  # Pokémon links
    names = [link.text.strip() for link in pokemon_links]
    return names

def parse_ev_yield(ev_text):
    """Convert '2 Sp. Atk, 1 Sp. Def' into a dict with numeric values."""
    evs = {stat: 0 for stat in STATS}
    if not ev_text:
        return evs
    
    parts = ev_text.split(",")
    for part in parts:
        part = part.strip()
        if part:
            try:
                num, stat = part.split(" ", 1)
                evs[stat] = int(num)
            except ValueError:
                pass
    return evs

def get_ev_yield(pokemon_name):
    # Map special names or convert to URL-friendly slug
    slug = SPECIAL_NAMES.get(pokemon_name, pokemon_name.lower().replace(" ", "-").replace("'", "").replace(".", ""))
    url = f"{BASE_URL}/pokedex/{slug}"
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ Failed to fetch {pokemon_name} ({url})")
        return {stat: 0 for stat in STATS}
    
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table", {"class": "vitals-table"})
    
    for table in tables:
        rows = table.find_all("tr")
        for row in rows:
            header = row.find("th")
            if header and "EV yield" in header.text:
                ev_yield = row.find("td").get_text(strip=True)
                return parse_ev_yield(ev_yield)
    
    return {stat: 0 for stat in STATS}

def main(limit=None):
    all_pokemon = get_all_pokemon_names()
    if limit:
        all_pokemon = all_pokemon[:limit]
    
    data = []
    for pokemon in all_pokemon:
        evs = get_ev_yield(pokemon)
        row = {"Pokemon": pokemon}
        row.update(evs)
        data.append(row)
        print(f"{pokemon} → {evs}")
        time.sleep(1)  # polite delay
    
    df = pd.DataFrame(data)
    df.to_csv("C:\\Users\\Brylle\Desktop\\Jupyter Projects\\Pokemon EV Scraper\\datapokemon_ev_yields.csv", index=False)
    print(f"\n✅ Data saved to pokemon_ev_yields.csv ({len(data)} Pokémon)")

if __name__ == "__main__":
    main(limit=0)  # Change/remove limit for full scrape


Bulbasaur → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 1, 'Sp. Def': 0, 'Speed': 0}
Ivysaur → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 1, 'Sp. Def': 1, 'Speed': 0}
Venusaur → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 2, 'Sp. Def': 1, 'Speed': 0}
Charmander → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 0, 'Sp. Def': 0, 'Speed': 1}
Charmeleon → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 1, 'Sp. Def': 0, 'Speed': 1}
Charizard → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 3, 'Sp. Def': 0, 'Speed': 0}
Squirtle → {'HP': 0, 'Attack': 0, 'Defense': 1, 'Sp. Atk': 0, 'Sp. Def': 0, 'Speed': 0}
Wartortle → {'HP': 0, 'Attack': 0, 'Defense': 1, 'Sp. Atk': 0, 'Sp. Def': 1, 'Speed': 0}
Blastoise → {'HP': 0, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 0, 'Sp. Def': 3, 'Speed': 0}
Caterpie → {'HP': 1, 'Attack': 0, 'Defense': 0, 'Sp. Atk': 0, 'Sp. Def': 0, 'Speed': 0}
Metapod → {'HP': 0, 'Attack': 0, 'Defense': 2, 'Sp. Atk': 0, 'Sp. Def': 0, 'Speed': 0}
Butterfree → {'HP': 0, 'At